<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/learnelixir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

メモ

elixir を齧る。かじる

* https://magazine.rubyist.net/articles/0054/0054-ElixirBook.html
* https://elixirschool.com/ja/lessons/basics/basics/